# Creating sample

This notebook shows how we can create a sample from atoms and calculate diffraction profiles using both constant wavelength and time-of-flight experiment types.

In [1]:
from easyCore import np
from easyDiffractionLib.sample import Sample
from easyDiffractionLib import Site, Phase, Phases
from easyDiffractionLib.interface import InterfaceFactory as Calculator

from easyDiffractionLib.Profiles.P1D import Instrument1DCWParameters as CWParams
from easyDiffractionLib.Profiles.P1D import Instrument1DTOFParameters as TOFParams
from easyDiffractionLib.Profiles.P1D import Powder1DParameters

from easyCore.Fitting.Fitting import Fitter

import matplotlib.pyplot as plt

**Preparing the sample and data**

In [ ]:
calculator = Calculator()
calculator.switch('CrysPy') # this is the default

Create an atom using `Site` interface

In [ ]:
atom = Site.from_pars(label="Cl1",
                      specie='Cl',
                      fract_x=0.0,
                      fract_y=0.0,
                      fract_z=0.0)

Create a phase

In [ ]:
phase = Phase(name="salt")

# set the space group
phase.spacegroup.space_group_HM_name = "F m -3 m"

# add the atom
phase.add_atom(atom)

Add another atom (using `Phase` interface)

In [ ]:
phase.add_atom('Na1', 'Na', 0.5, 0.5, 0.5)

Visualise the structure

In [ ]:
import py3Dmol
viewer = py3Dmol.view()
viewer.addModel(phase.to_cif_str(),'cif',{'doAssembly':True,'duplicateAssemblyAtoms':True,'normalizeAssembly':True})
viewer.setStyle({'sphere':{'colorscheme':'Jmol','scale':.2},'stick':{'colorscheme':'Jmol', 'radius': 0.1}})
viewer.addUnitCell()
viewer.replicateUnitCell(2,2,2)
#viewer.setStyle({'sym':2},{'sphere':{'scale':.5,'color':'blue'},'stick':{'color':'cyan'}})
viewer.zoomTo()

Create Phases object

In [ ]:
phases = Phases()
phases.append(phase)

Create and modify a Parameters object

In [ ]:
parameters = CWParams.default()

parameters.length_a = 5.6402
parameters.resolution_u = 0.1447
parameters.resolution_v = -0.4252
parameters.resolution_w = 0.3864
parameters.resolution_x = 0.0
parameters.resolution_y = 0.0

Create and modify a Pattern object

In [ ]:
pattern = Powder1DParameters.default()
pattern.zero_shift = 0.0
pattern.scale = 100.0

Finally, create the Sample

In [ ]:
S = Sample(phases=phases, parameters=parameters, calculator=calculator)

Calculate the profile - using the calculator we defined previously.
Note that we don't run anything on the `Sample` object but on a separate `Calculator` instance

In [ ]:
x_data = np.linspace(1, 120, 500)
y_data = calculator.fit_func(x_data)

%matplotlib notebook
plt.plot(x_data, y_data, label='CW Simulation')
plt.legend()

## Switch to TOF

Retrieve the phases and pattern

In [ ]:
phases = S.phases[0]
pattern = S.pattern

Modify the parameters to correspond to a TOF experiment

In [ ]:
parameters = TOFParams.default()
parameters.length_a = 5.6402
parameters.dtt1 = 6167.24700
parameters.dtt2 = -2.28000
parameters.ttheta_bank = 145.00

In [ ]:
pattern.zero_shift = 0.0
pattern.scale = 100.0

tof_str = 'Npowder1DTOF'
interfaces = calculator.interface_compatability(tof_str)
print(interfaces[0])
calculator.switch(interfaces[0])

S = Sample(phases=phases, parameters=parameters, calculator=calculator)


Calculate the profile again, this time based on the TOF parameters

In [ ]:
x_data = np.linspace(5000, 60000, 500)
y_data = calculator.fit_func(x_data)

%matplotlib notebook

plt.plot(x_data, y_data, label='TOF Simulation')
plt.legend()